In [74]:
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import torch
import pandas as pd

In [75]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")

In [76]:
X = np.load('Data\\X.npy', allow_pickle=True)
y = np.load('Data\\y.npy', allow_pickle=True)

In [77]:
#y_oh = np.zeros((y.shape[0], max(y)+1))
#y_oh[np.arange(y.shape[0]), y] = 1

In [78]:
random.seed(0)
np.random.seed(0)
idx = np.arange(X.shape[0])
random.shuffle(idx)  
btra = np.random.choice(idx, int(0.8*X.shape[0]), replace=False)
btes = [i for i in idx if i not in btra]

In [79]:
Xtra = X[btra]
ytra = y[btra]

Xtes = X[btes]
ytes = y[btes]

In [80]:
Xtes.shape

(129, 468)

In [81]:
def batch_generator(X, Y, n_batches):  
    
    random.seed(123)
    
    batch_size = X.shape[0] // n_batches
    
    idx = np.arange(X.shape[0])
    random.shuffle(idx)    
    idx = idx[:n_batches*batch_size]
        
    for i in range(n_batches):            
        bi = np.random.choice(idx, batch_size, replace=False)
        X_batch = X[bi]
        Y_batch = Y[bi]
        idx = [i for i in idx if i not in bi]
        yield (X_batch,Y_batch)       
        
        
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(468, 24, bias=True)
        self.fc12 = nn.Linear(24, 13, bias=True) 
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)     
        return x1
    
    
def fit(X, Y, Xt, Yt, net, optimizer, error, n_epochs, n_batches, device, PATH, min_val_loss = float('inf')):
    
    net = net.to(device)    
    losses = []    
    val_losses = []

    val_inputs = torch.FloatTensor(Xt)
    val_labels = torch.tensor(Yt, dtype=torch.long)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)  
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
    
        running_loss = 0  
         
        
        for batch_x, batch_y in batch_generator(X, Y, n_batches):  
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # get the inputs
            inputs = torch.FloatTensor(batch_x)
            labels = torch.tensor(batch_y, dtype=torch.long)
            inputs, labels = inputs.to(device), labels.to(device)             
                
    
            # forward + backward + optimize
            outputs = net.forward(inputs)
            loss = error(outputs, labels)
                        
            loss.backward()    #obtain gradients      
            optimizer.step()   #optimize
                
            running_loss += loss.item()      
                
        running_loss = running_loss/n_batches    
        
        with torch.no_grad():
            val_outputs = net.forward(val_inputs)
            val_loss = error(val_outputs, val_labels) 
        
        losses.append(running_loss)   
        val_losses.append(val_loss.item())
        
        
        print('Epoch {0}: Training Loss: {1} Validation Loss: {2}'.format(epoch+1, running_loss, val_loss.item()))
        
        if val_loss.item() < min_val_loss:
            torch.save(net.state_dict(), PATH)
            print('New Checkpoint Saved into PATH')
            min_val_loss = val_loss.item()
        
        
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)        
        

In [82]:
n_epochs = 10000
lr = 0.001
n_batches = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = 'Data\\model_checkpoint.pt'

In [83]:
net = Net1()
net.apply(weights_init_normal)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [84]:
fit(Xtra, ytra, Xtes, ytes, net, optimizer, criterion, n_epochs, n_batches, device, PATH)

Epoch 1: Training Loss: 2.540684143702189 Validation Loss: 2.554556131362915
New Checkpoint Saved into PATH
Epoch 2: Training Loss: 2.5284109115600586 Validation Loss: 2.533421516418457
New Checkpoint Saved into PATH
Epoch 3: Training Loss: 2.5106197198232016 Validation Loss: 2.5088155269622803
New Checkpoint Saved into PATH
Epoch 4: Training Loss: 2.49190886815389 Validation Loss: 2.4839248657226562
New Checkpoint Saved into PATH
Epoch 5: Training Loss: 2.4714925289154053 Validation Loss: 2.4592032432556152
New Checkpoint Saved into PATH
Epoch 6: Training Loss: 2.4543023109436035 Validation Loss: 2.434687852859497
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 2.434699773788452 Validation Loss: 2.4093384742736816
New Checkpoint Saved into PATH
Epoch 8: Training Loss: 2.4169743061065674 Validation Loss: 2.384373188018799
New Checkpoint Saved into PATH
Epoch 9: Training Loss: 2.4014358520507812 Validation Loss: 2.3609113693237305
New Checkpoint Saved into PATH
Epoch 10: Training

New Checkpoint Saved into PATH
Epoch 80: Training Loss: 2.196745236714681 Validation Loss: 2.135190486907959
New Checkpoint Saved into PATH
Epoch 81: Training Loss: 2.194838364919027 Validation Loss: 2.1341910362243652
New Checkpoint Saved into PATH
Epoch 82: Training Loss: 2.193137248357137 Validation Loss: 2.13313364982605
New Checkpoint Saved into PATH
Epoch 83: Training Loss: 2.1915839513142905 Validation Loss: 2.1334245204925537
Epoch 84: Training Loss: 2.1895345052083335 Validation Loss: 2.1308836936950684
New Checkpoint Saved into PATH
Epoch 85: Training Loss: 2.1876839796702066 Validation Loss: 2.128386974334717
New Checkpoint Saved into PATH
Epoch 86: Training Loss: 2.185959736506144 Validation Loss: 2.1264119148254395
New Checkpoint Saved into PATH
Epoch 87: Training Loss: 2.1845359802246094 Validation Loss: 2.1234521865844727
New Checkpoint Saved into PATH
Epoch 88: Training Loss: 2.182447671890259 Validation Loss: 2.122577667236328
New Checkpoint Saved into PATH
Epoch 89: T

Epoch 162: Training Loss: 2.0493961175282798 Validation Loss: 2.00840163230896
New Checkpoint Saved into PATH
Epoch 163: Training Loss: 2.047572294871012 Validation Loss: 2.007171869277954
New Checkpoint Saved into PATH
Epoch 164: Training Loss: 2.045844554901123 Validation Loss: 2.0075089931488037
Epoch 165: Training Loss: 2.043759822845459 Validation Loss: 2.007556676864624
Epoch 166: Training Loss: 2.0419774850209556 Validation Loss: 2.0049819946289062
New Checkpoint Saved into PATH
Epoch 167: Training Loss: 2.040222446123759 Validation Loss: 2.001438617706299
New Checkpoint Saved into PATH
Epoch 168: Training Loss: 2.0382893880208335 Validation Loss: 1.9991812705993652
New Checkpoint Saved into PATH
Epoch 169: Training Loss: 2.036406993865967 Validation Loss: 1.998136281967163
New Checkpoint Saved into PATH
Epoch 170: Training Loss: 2.0348229010899863 Validation Loss: 1.9972248077392578
New Checkpoint Saved into PATH
Epoch 171: Training Loss: 2.0332559744517007 Validation Loss: 1.9

New Checkpoint Saved into PATH
Epoch 247: Training Loss: 1.8946079810460408 Validation Loss: 1.8792145252227783
Epoch 248: Training Loss: 1.8930145104726155 Validation Loss: 1.8800114393234253
Epoch 249: Training Loss: 1.8911550839742024 Validation Loss: 1.878274917602539
New Checkpoint Saved into PATH
Epoch 250: Training Loss: 1.8896323839823406 Validation Loss: 1.8705288171768188
New Checkpoint Saved into PATH
Epoch 251: Training Loss: 1.8876090447107952 Validation Loss: 1.8718106746673584
Epoch 252: Training Loss: 1.885483702023824 Validation Loss: 1.869613528251648
New Checkpoint Saved into PATH
Epoch 253: Training Loss: 1.8833419879277546 Validation Loss: 1.8700650930404663
Epoch 254: Training Loss: 1.8817482391993205 Validation Loss: 1.869027018547058
New Checkpoint Saved into PATH
Epoch 255: Training Loss: 1.8800022602081299 Validation Loss: 1.8685332536697388
New Checkpoint Saved into PATH
Epoch 256: Training Loss: 1.8780768314997356 Validation Loss: 1.8655502796173096
New Chec

Epoch 339: Training Loss: 1.7334162791570027 Validation Loss: 1.738683819770813
Epoch 340: Training Loss: 1.7302478949228923 Validation Loss: 1.7463558912277222
Epoch 341: Training Loss: 1.7298332452774048 Validation Loss: 1.7485135793685913
Epoch 342: Training Loss: 1.7274491786956787 Validation Loss: 1.7402626276016235
Epoch 343: Training Loss: 1.7250785827636719 Validation Loss: 1.7331318855285645
New Checkpoint Saved into PATH
Epoch 344: Training Loss: 1.7237000465393066 Validation Loss: 1.733264684677124
Epoch 345: Training Loss: 1.721915324529012 Validation Loss: 1.7327206134796143
New Checkpoint Saved into PATH
Epoch 346: Training Loss: 1.7207426230112712 Validation Loss: 1.7345166206359863
Epoch 347: Training Loss: 1.7184545596440632 Validation Loss: 1.733074426651001
Epoch 348: Training Loss: 1.7171351512273152 Validation Loss: 1.7292619943618774
New Checkpoint Saved into PATH
Epoch 349: Training Loss: 1.7149675687154133 Validation Loss: 1.7290676832199097
New Checkpoint Saved

...

Epoch 9498: Training Loss: 0.052476865549882255 Validation Loss: 0.7811408638954163
Epoch 9499: Training Loss: 0.052366302659114204 Validation Loss: 0.7785940766334534
Epoch 9500: Training Loss: 0.05262211958567301 Validation Loss: 0.7762192487716675
Epoch 9501: Training Loss: 0.05238805462916692 Validation Loss: 0.781505823135376
Epoch 9502: Training Loss: 0.052309410025676094 Validation Loss: 0.7810519933700562
Epoch 9503: Training Loss: 0.05224800234039625 Validation Loss: 0.7845028638839722
Epoch 9504: Training Loss: 0.0523051992058754 Validation Loss: 0.7831382751464844
Epoch 9505: Training Loss: 0.05265961214900017 Validation Loss: 0.7789732217788696
Epoch 9506: Training Loss: 0.05229418848951658 Validation Loss: 0.7815243005752563
Epoch 9507: Training Loss: 0.05231118698914846 Validation Loss: 0.7814579606056213
Epoch 9508: Training Loss: 0.052199924985567726 Validation Loss: 0.7818201780319214
Epoch 9509: Training Loss: 0.05236684655149778 Validation Loss: 0.784636914730072
Epo

Epoch 9605: Training Loss: 0.051882073283195496 Validation Loss: 0.783781111240387
Epoch 9606: Training Loss: 0.051782477647066116 Validation Loss: 0.7845754027366638
Epoch 9607: Training Loss: 0.05191966270407041 Validation Loss: 0.7815781235694885
Epoch 9608: Training Loss: 0.05184552570184072 Validation Loss: 0.7824870944023132
Epoch 9609: Training Loss: 0.05185182144244512 Validation Loss: 0.7826075553894043
Epoch 9610: Training Loss: 0.05195979277292887 Validation Loss: 0.7843950986862183
Epoch 9611: Training Loss: 0.05183834210038185 Validation Loss: 0.7826080322265625
Epoch 9612: Training Loss: 0.05204059680302938 Validation Loss: 0.7839012145996094
Epoch 9613: Training Loss: 0.051799570520718895 Validation Loss: 0.779128909111023
Epoch 9614: Training Loss: 0.051829143116871514 Validation Loss: 0.777005672454834
Epoch 9615: Training Loss: 0.051877407977978386 Validation Loss: 0.7787295579910278
Epoch 9616: Training Loss: 0.051784822096427284 Validation Loss: 0.7804779410362244
E

Epoch 9711: Training Loss: 0.05143774921695391 Validation Loss: 0.7853648066520691
Epoch 9712: Training Loss: 0.05138947876791159 Validation Loss: 0.7832548022270203
Epoch 9713: Training Loss: 0.051428026209274925 Validation Loss: 0.7801632881164551
Epoch 9714: Training Loss: 0.05144695440928141 Validation Loss: 0.7813801765441895
Epoch 9715: Training Loss: 0.051483084758122764 Validation Loss: 0.7829292416572571
Epoch 9716: Training Loss: 0.051515584190686546 Validation Loss: 0.7808459997177124
Epoch 9717: Training Loss: 0.05134927108883858 Validation Loss: 0.7824082970619202
Epoch 9718: Training Loss: 0.05135813479622205 Validation Loss: 0.7834600806236267
Epoch 9719: Training Loss: 0.05143788208564123 Validation Loss: 0.783959150314331
Epoch 9720: Training Loss: 0.051465440541505814 Validation Loss: 0.7823780179023743
Epoch 9721: Training Loss: 0.051501130064328514 Validation Loss: 0.7815254926681519
Epoch 9722: Training Loss: 0.05142862722277641 Validation Loss: 0.7839332222938538


Epoch 9817: Training Loss: 0.051325382043917976 Validation Loss: 0.7818657755851746
Epoch 9818: Training Loss: 0.05104765544335047 Validation Loss: 0.7811182737350464
Epoch 9819: Training Loss: 0.051028323670228325 Validation Loss: 0.7841727137565613
Epoch 9820: Training Loss: 0.05100066587328911 Validation Loss: 0.7859430313110352
Epoch 9821: Training Loss: 0.05100497975945473 Validation Loss: 0.7865669131278992
Epoch 9822: Training Loss: 0.05150904506444931 Validation Loss: 0.7814955115318298
Epoch 9823: Training Loss: 0.05111895874142647 Validation Loss: 0.7832939028739929
Epoch 9824: Training Loss: 0.0511485810081164 Validation Loss: 0.782336413860321
Epoch 9825: Training Loss: 0.05096366504828135 Validation Loss: 0.7832857370376587
Epoch 9826: Training Loss: 0.05104194954037666 Validation Loss: 0.7833330035209656
Epoch 9827: Training Loss: 0.05135221406817436 Validation Loss: 0.7893552780151367
Epoch 9828: Training Loss: 0.05109539503852526 Validation Loss: 0.7901492714881897
Epoc

Epoch 9919: Training Loss: 0.05083914225300153 Validation Loss: 0.7839931845664978
Epoch 9920: Training Loss: 0.05064661428332329 Validation Loss: 0.7827274799346924
Epoch 9921: Training Loss: 0.05073627457022667 Validation Loss: 0.7839576601982117
Epoch 9922: Training Loss: 0.050690912952025734 Validation Loss: 0.785149872303009
Epoch 9923: Training Loss: 0.05071237434943517 Validation Loss: 0.7873651385307312
Epoch 9924: Training Loss: 0.05069355418284734 Validation Loss: 0.7852203249931335
Epoch 9925: Training Loss: 0.05087004601955414 Validation Loss: 0.7826641201972961
Epoch 9926: Training Loss: 0.050653244058291115 Validation Loss: 0.7823081612586975
Epoch 9927: Training Loss: 0.05063389986753464 Validation Loss: 0.7839925289154053
Epoch 9928: Training Loss: 0.050984833389520645 Validation Loss: 0.7895542979240417
Epoch 9929: Training Loss: 0.0506106565395991 Validation Loss: 0.7895016670227051
Epoch 9930: Training Loss: 0.05058359851439794 Validation Loss: 0.7874559760093689
Epo

In [85]:
PATH = 'Data\\model_checkpoint.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [86]:
error = nn.CrossEntropyLoss()

In [87]:
with torch.no_grad():
    net.eval()
    inputs = torch.FloatTensor(Xtes)
    labels = torch.tensor(ytes, dtype=torch.long)
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net.forward(inputs)
    loss = error(outputs, labels) 
    print(loss.item())

0.7188012003898621


In [64]:
param_dict = {}
for v,p in enumerate(net.parameters()):
    param_dict[v] = p.data.numpy()
    
W0 = param_dict[0].T
b0 = param_dict[1]
W1 = param_dict[2].T
b1 = param_dict[3]


h0 = np.matmul(Xtes, W0) + b0
h1 = np.tanh(h0)
h2 = np.matmul(h1, W1) + b1
h3 = np.exp(h2)
o = h3/np.sum(h3,axis=1).reshape(-1,1)

In [65]:
def y_hat(prob, topics):    
    return topics[np.argmax(prob)]
    

In [66]:
pred_labels = []
topics = ['Jubilacion Patronal', 'Consultoria', 'Renuncia/Despido/Desahucio', 'IESS', 
                 'Greeting', 'Contacto', 'No Topic', 'Queja', 'Otros servicios', 'Charlas/Capacitaciones', 
                      'Hi Five', 'job seeker', 'Facturacion/Retencion/Cobros']
for c in o:
    pred_labels.append(y_hat(c, topics))

In [68]:
preguntas = data.Pregunta[btes]
labels = data.Tema[btes]


In [69]:
Xtes = np.column_stack((preguntas, labels, pred_labels, Xtes, o))


In [70]:
data2 = pd.DataFrame(Xtes)


In [71]:
vocab = np.load('Data\\vocab.npy', allow_pickle=True)
colnames = np.concatenate((['pregunta', 'label', 'pred_label'], vocab, ['n_token', 'perc_greet', 'polarity'], ['Jubilacion Patronal', 'Consultoria', \
                                                                                                     'Renuncia/Despido/Desahucio', 'IESS', 'Greeting', \
                                                                                                         'Contacto', 'No Topic', 'Queja', 'Otros servicios', \
                                                                                                 'Charlas/Capacitaciones', 'Hi Five', 'job seeker', 'Facturacion/Retencion/Cobros']))
data2.columns = colnames


In [72]:
data2.head(3)


pregunta  \
0   Necesito informacion sobre su producto ACTUA R2O   
1  Buenos dias tengo dudas en cuanto a los servic...   
2               Liquidación por desahucio por 7 años   

                        label                  pred_label hol una pregunt en  \
0                 Consultoria                 Consultoria   0   0       0  0   
1                    Contacto                    Contacto   0   0       0  1   
2  Renuncia/Despido/Desahucio  Renuncia/Despido/Desahucio   0   0       0  0   

  que perjudic el  ...        IESS     Greeting     Contacto     No Topic  \
0   0        0  0  ...  0.00122106   0.00256981   0.00148832    0.0257649   
1   1        0  0  ...   0.0292173   0.00352382     0.503212   0.00817231   
2   0        0  0  ...  0.00118378  0.000490635  5.81032e-05  0.000960213   

         Queja Otros servicios Charlas/Capacitaciones     Hi Five  \
0   0.00512395        0.138984              0.0159844  0.00304579   
1     0.100545       0.0475307              0.0319105   0.0536152   
2  0.000626847     0.000639163             0.00037006  0.00155689   

    job seeker Facturacion/Retencion/Cobros  
0   0.00412263                   0.00121621  
1     0.048752                    0.0778633  
2  0.000649705                  0.000110804  

[3 rows x 484 columns]

In [73]:
data2.to_csv('Data\\Xtes.csv', sep=',')